In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sentence_transformers import util
import torch

# --- Загрузка данных ---
print("📁 Загрузка датасета...")
df = pd.read_csv("recipes_ru.csv")

def enhance_recipe_text(row):
    name = row['name'] if pd.notna(row['name']) else ''
    description = row['description'] if pd.notna(row['description']) else ''
    ingredients = row['ingredients'] if pd.notna(row['ingredients']) else ''
    instructions = row['instructions'] if pd.notna(row['instructions']) else ''

    # Увеличиваем приоритет названия и описания
    enhanced_text = ((name + ' ') * 3 + (description + ' ') * 2 + ingredients + ' ' + instructions).strip().lower()
    
    return enhanced_text

# --- Создание нового поля с улучшенным текстом ---
print("🧹 Обработка рецептов...")
df['enhanced_text'] = df.apply(enhance_recipe_text, axis=1)
recipe_texts = df["enhanced_text"].tolist()

# --- Загрузка модели SBERT ---
print("🧠 Загрузка модели...")
model = SentenceTransformer('cointegrated/rubert-tiny2')

# --- Получение эмбеддингов ---
print("🧬 Кодирование рецептов...")
recipe_embeddings = model.encode(recipe_texts, show_progress_bar=True)

# --- Функция поиска ---
def find_similar_recipes(query, top_n=5):
    query_embedding = model.encode([query])
    cos_scores = util.cos_sim(query_embedding, recipe_embeddings)[0]
    top_indices = torch.topk(cos_scores, k=top_n).indices.tolist()
    
    results = []
    for idx in top_indices:
        results.append({
            "name": df.iloc[idx]["name"],
            "ingredients": df.iloc[idx]["ingredients"],
            "description": df.iloc[idx]["description"],
            "score": cos_scores[idx].item()
        })
    return results

# --- Пример использования ---
query = input("Введите ваш запрос: ")
print(f"\n🔍 Результаты по запросу: '{query}'")
results = find_similar_recipes(query)

for res in results:
    # print(f"\n{res['name']} | Score: {res['score']:.4f}")
    print(f"\n{res['name']}")
    print(f"Ингредиенты: {res['ingredients']}")
    # print(f"Описание: {res['description']}")

c:\Users\mrcov\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📁 Загрузка датасета...
🧹 Обработка рецептов...
🧠 Загрузка модели...
🧬 Кодирование рецептов...


Batches: 100%|██████████| 3/3 [00:01<00:00,  1.75it/s]



🔍 Результаты по запросу: 'Свинина'

Пикантный стейк-спрей
Ингредиенты: 1/4 стакана сладкого вермута, 1/4 стакана бренди, 1 столовая ложка жидкого дымного ароматизатора, 1/8 стакана Kitchen Bouquet, 1/2 стакана вустерширского соуса, 1 чайная ложка приправы для стейка, 1 чайная ложка приправленной соли, 1/2 чайной ложки молотого черного перца

Волшебное зелье Шелби
Ингредиенты: 1/2 стакана белого сахара, 1 пакет несладкого фруктового порошка для напитков, 1 литр лимонно-лаймовой газировки, 1/2 стакана холодной воды, 1 кварта ванильного мороженого

Курица с клецками в медленноварке
Ингредиенты: 4 куриных грудки без кожи и костей, 2 столовые ложки сливочного масла, 2 банки сгущенного куриного супа, 1 луковица мелко нарезанная, 2 упаковки охлажденного теста для булочек разорванного на кусочки

Грузинский закат
Ингредиенты: 1 рюмка бренди, 1 рюмка персикового шнапса, 1 унция сиропа гренадин, 1/4 стакана охлажденной лимонно-лаймовой газировки

Нью-йоркский чизкейк от Шанталь
Ингредиенты: 15 